In [ ]:
import pandas as pd
from ensemble_boxes import *

In [ ]:
# csv 파일 로드
file1 = '/opt/ml/DACON-objectdetection/mmdetection/work_dirs/01_swin_bbox_safe/submission_best_bbox_mAP_epoch_16.csv'
file2 = '/opt/ml/DACON-objectdetection/mmdetection/work_dirs/cascade-mask-rcnn_convnext-t/submission_best_bbox_mAP_epoch_20.csv'
file3 = '/opt/ml/DACON-objectdetection/mmdetection/work_dirs/02_res2net101_bbox_safe/submission_best_bbox_mAP_epoch_16.csv'
output_file = '/opt/ml/DACON-objectdetection/results/ensemble.csv'

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)

df1['source'] = file1
df2['source'] = file2
df3['source'] = file3


the_weight = [0.4, 0.3, 0.3]    # wbf ensemble 비율 설정


# 세 DataFrame을 하나로 결합
df = pd.concat([df1, df2, df3])

# 이미지의 너비와 높이
IMAGE_WIDTH = 1920
IMAGE_HEIGHT = 1080

def convert_boxes(row):
    return [(row['point1_x'] / IMAGE_WIDTH), (row['point1_y'] / IMAGE_HEIGHT), 
            (row['point3_x'] / IMAGE_WIDTH), (row['point3_y'] / IMAGE_HEIGHT)]


df['boxes'] = df.apply(lambda row: convert_boxes(row), axis=1)